In [6]:
# load xgboost_model_with_counter.pkl

import pickle
import xgboost as xgb
import numpy as np

# load model
xgb_model = pickle.load(open("xgboost_model_with_counter.pkl", "rb"))

In [7]:
# load features_with_label_df_with_counter.pkl 
features_with_label_df_with_counter = pickle.load(open("features_with_label_df_with_counter.pkl", "rb"))

In [8]:
features_with_label_df_with_counter.head()

,minutes,score_difference,distance_to_goal,angle_to_goal,foul_count_player,foul_count_team,vaep_offensive,label,freeze_frame,match_id,vertical_range,teammate_count,opponent_count,keeper_visible
id,,,,,,,,,,,,,,
f2bd21f6-87c3-44c0-bb37-04e996db413e,23,1,65.011153,0.058485,1,1,-0.001101,0,"[{'teammate': True, 'actor': False, 'keeper': ...",3857256,"[55.099998474121094, 120]",3,7,0
8e51baec-6948-4285-86a1-73197126e265,33,0,69.641654,-0.306311,1,2,-0.001289,1,"[{'teammate': True, 'actor': False, 'keeper': ...",3857256,"[0, 66.4000015258789]",6,5,0
b711f529-182e-4ae3-8c9a-189b7697b2ea,46,0,52.106142,-0.264064,1,2,-0.000542,1,"[{'teammate': True, 'actor': False, 'keeper': ...",3857256,"[69.69999694824219, 120]",3,4,0
ec46c71c-af4c-4d99-a854-e4ffc30460d6,55,1,42.569942,0.554117,1,3,-0.013511,1,"[{'teammate': True, 'actor': False, 'keeper': ...",3857256,"[83.80000305175781, 120]",6,10,0
142138e3-20c9-46b2-80cd-db92a583bc81,80,1,98.521165,0.092498,1,5,0.001283,1,"[{'teammate': True, 'actor': False, 'keeper': ...",3857256,"[21.899999618530273, 120]",4,5,1


In [9]:
# for each row in features_with_label_df_with_counter, predict probability from model

# create a new column in features_with_label_df_with_counter called 'probability'
features_with_label_df_with_counter['probability'] = 0.0

# use columns: 'minutes', 'score_difference', 'distance_to_goal', 'angle_to_goal', 'foul_count_player', 'foul_count_team', 'vaep_offensive', 'teammate_count', 'opponent_count' as features
features = ['minutes', 'score_difference', 'distance_to_goal', 'angle_to_goal', 'foul_count_player', 'foul_count_team', 'vaep_offensive', 'teammate_count', 'opponent_count']

# for each row in features_with_label_df_with_counter, predict probability from model

for index, row in features_with_label_df_with_counter.iterrows():
    # get features
    x = row[features].values
    # reshape
    x = x.reshape(1, -1)
    
    # predict probability
    y = xgb_model.predict_proba(x)
    # update probability column
    features_with_label_df_with_counter.at[index, 'probability'] = y[0][1]

In [31]:
# make a new df from features_with_label_df_with_counter with columns: 'match_id', 'label', 'probability'. keep the same index
df = features_with_label_df_with_counter[['match_id', 'label', 'probability']]

In [32]:
from statsbombpy import sb
# add columns 'team_id', 'team', 'player', 'player_id' to df
df.loc[:, 'team_id'] = None
df.loc[:, 'team'] = None
df.loc[:, 'player'] = None
df.loc[:, 'player_id'] = None

for index, row in df.iterrows():
    # make match_id a int
    match_id = int(row['match_id'])
    event = sb.events(match_id=match_id)

    # get the event where index matches the 'id' column of the row
    event = event[event['id'] == index]

    # get 'team_id', 'team', 'player', 'player_id'
    team_id = event['team_id']
    team = event['team']
    player = event['player']
    player_id = event['player_id']

    # add team_id, team, player, player_id to df
    df.at[index, 'team_id'] = team_id
    df.at[index, 'team'] = team
    df.at[index, 'player'] = player
    df.at[index, 'player_id'] = player_id

/tmp/ipykernel_3841358/3114590715.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'team_id'] = None
/tmp/ipykernel_3841358/3114590715.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'team'] = None
/tmp/ipykernel_3841358/3114590715.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/inde

In [38]:
# in df, change 'team_id', 'player_id' to int
df['team_id'] = df['team_id'].astype(int)
df['player_id'] = df['player_id'].astype(int)

# In df, group all player_id. For each player_id, get the sum of 'label' and the sum of 'probability'. Make player_id the index. Keep player column
players = df.groupby('player_id').agg({'label': 'sum', 'probability': 'sum', 'player': 'first'})

teams = df.groupby('team_id').agg({'label': 'sum', 'probability': 'sum', 'team': 'first'})


/tmp/ipykernel_3841358/775135970.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['team_id'] = df['team_id'].astype(int)
/tmp/ipykernel_3841358/775135970.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['player_id'] = df['player_id'].astype(int)


In [50]:
# sort players by probability
players = players.sort_values(by=['probability'], ascending=False)

# sort players by ratio of probability to label in ascending order
players['ratio'] = players['probability'] / players['label']
players = players.sort_values(by=['ratio'], ascending=True)

players[players['label'] > 1].tail()

,label,probability,player,ratio
player_id,,,,
10658,2,1.982030,"4395 Danielle van de Donk Name: player, dty...",0.991015
19597,3,2.973920,"3838 Marcos Javier Acuña Name: player, dtyp...",0.991307
4353,2,1.986347,"4194 Aymeric Laporte Name: player, dtype: o...",0.993173
131694,2,1.989330,"4095 Martha Tembo Name: player, dtype: object",0.994665
5549,2,2.932505,"3310 Manuel Obafemi Akanji Name: player, dt...",1.466253


In [55]:
# sort players by probability
teams = teams.sort_values(by=['probability'], ascending=False)

# sort players by ratio of probability to label in ascending order
teams['ratio'] = teams['probability'] / teams['label']
teams = teams.sort_values(by=['ratio'], ascending=True)

teams[teams['label'] > 1].tail()

,label,probability,team,ratio
team_id,,,,
865,8,9.693448,"4041 England Women's Name: team, dtype: object",1.211681
860,5,6.132399,"3297 Switzerland Women's Name: team, dtype:...",1.226480
1206,3,3.834055,"2781 Canada Women's Name: team, dtype: object",1.278018
941,5,6.536445,"3845 Netherlands Name: team, dtype: object",1.307289
908,3,4.010371,"3508 Slovakia Name: team, dtype: object",1.336790
